# Diffusion of a Gaussian Function on a Circular Domain

Author: Jørgen S. Dokken (modified for circular domain)

This notebook solves the diffusion of a Gaussian hill on a circular domain (disk centered at (0,0) with radius 2). The initial condition is:
\begin{align}
    u_0(x,y) = e^{-5x^2 - 5y^2}
\end{align}
We use homogeneous Dirichlet boundary conditions ($u = 0$) on the circular boundary. The domain is meshed with quadrilateral-dominated elements using `gmsh`.

The mesh is generated using `gmsh`, and the problem is solved using DOLFINx with a finite element method.

In [1]:
import matplotlib as mpl
import pyvista
import ufl
import numpy as np
from petsc4py import PETSc
from mpi4py import MPI
from dolfinx import fem, mesh, io, plot
from dolfinx.fem.petsc import assemble_vector, assemble_matrix, create_vector, apply_lifting, set_bc
import gmsh
from dolfinx.io import gmshio

# Define temporal parameters
t = 0
T = 1.0
num_steps = 50
dt = T / num_steps

# Generate circular mesh with gmsh
gmsh.initialize()
gmsh.model.add("disk")
gmsh.model.occ.addDisk(0, 0, 0, 2, 2)  # Disk centered at (0,0), radius 2
gmsh.model.occ.synchronize()
gmsh.model.addPhysicalGroup(2, [1], tag=1)  # Mark surface
gmsh.model.addPhysicalGroup(1, [1], tag=2)  # Mark boundary
gmsh.option.setNumber("Mesh.RecombineAll", 1)  # Encourage quadrilateral elements
gmsh.option.setNumber("Mesh.Algorithm", 8)  # Use frontal algorithm for quads
gmsh.model.mesh.setSize(gmsh.model.getEntities(0), 0.08)  # Approximate element size
gmsh.model.mesh.generate(2)
gmsh_model_rank = 0
mesh_comm = MPI.COMM_WORLD
domain, _, _ = gmshio.model_to_mesh(gmsh.model, mesh_comm, gmsh_model_rank, gdim=2)
gmsh.finalize()

# Define function space
V = fem.functionspace(domain, ("Lagrange", 1))

The circular domain is a disk of radius 2, meshed with quadrilateral-dominated elements for better accuracy on regular geometries. The mesh resolution is controlled by the element size in `gmsh`.

In [2]:
# Create initial condition
def initial_condition(x, a=5):
    return np.exp(-a * (x[0]**2 + x[1]**2))

u_n = fem.Function(V)
u_n.name = "u_n"
u_n.interpolate(initial_condition)

# Create boundary condition (Dirichlet on entire circular boundary)
fdim = domain.topology.dim - 1
boundary_facets = mesh.locate_entities_boundary(
    domain, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
bc = fem.dirichletbc(PETSc.ScalarType(0), fem.locate_dofs_topological(V, fdim, boundary_facets), V)

## Time-Dependent Output
We use an `XDMFFile` to store the solution for visualization in Paraview, storing the mesh once and appending solutions to reduce file size.

In [3]:
xdmf = io.XDMFFile(domain.comm, "diffusion_circular.xdmf", "w")
xdmf.write_mesh(domain)

# Define solution variable
uh = fem.Function(V)
uh.name = "uh"
uh.interpolate(initial_condition)
xdmf.write_function(uh, t)

## Variational Problem and Solver
Define the variational problem for the diffusion equation.

In [4]:
u, v = ufl.TrialFunction(V), ufl.TestFunction(V)
f = fem.Constant(domain, PETSc.ScalarType(0))
a = u * v * ufl.dx + dt * ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx
L = (u_n + dt * f) * v * ufl.dx

## Preparing Linear Algebra Structures
Assemble the bilinear and linear forms.

In [5]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)

Assemble the matrix once and create a reusable vector for the right-hand side.

In [6]:
A = assemble_matrix(bilinear_form, bcs=[bc])
A.assemble()
b = create_vector(linear_form)

## Linear Solver
Set up a PETSc KSP solver with LU factorization.

In [7]:
solver = PETSc.KSP().create(domain.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

## Visualization with PyVista
Visualize the solution as a GIF using PyVista.

In [8]:
pyvista.start_xvfb()
grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(V))
plotter = pyvista.Plotter()
plotter.open_gif("u_time_circular.gif", fps=10)
grid.point_data["uh"] = uh.x.array
warped = grid.warp_by_scalar("uh", factor=1)
viridis = mpl.colormaps.get_cmap("viridis").resampled(25)
sargs = dict(title_font_size=25, label_font_size=20, fmt="%.2e", color="black",
             position_x=0.1, position_y=0.8, width=0.8, height=0.1)
renderer = plotter.add_mesh(warped, show_edges=True, lighting=False,
                            cmap=viridis, scalar_bar_args=sargs,
                            clim=[0, max(uh.x.array)])

## Time-Stepping Loop
Solve the problem at each time step, updating the solution and visualization.

In [9]:
for i in range(num_steps):
    t += dt
    with b.localForm() as loc_b:
        loc_b.set(0)
    assemble_vector(b, linear_form)
    apply_lifting(b, [bilinear_form], [[bc]])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    set_bc(b, [bc])
    solver.solve(b, uh.x.petsc_vec)
    uh.x.scatter_forward()
    u_n.x.array[:] = uh.x.array
    xdmf.write_function(uh, t)
    new_warped = grid.warp_by_scalar("uh", factor=1)
    warped.points[:, :] = new_warped.points
    warped.point_data["uh"][:] = uh.x.array
    plotter.write_frame()
plotter.close()
xdmf.close()

<img src="./u_time_circular.gif" alt="gif" class="bg-primary mb-1" width="800px">

## Animation with Paraview
Open the `diffusion_circular.xdmf` file in Paraview, apply settings, add time annotation, and save the animation as described in the original notebook.